<a href="https://colab.research.google.com/github/FastBackendML/FastBackendML/blob/main/Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.10 Использование CPU и GPU на примере простой НС

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
t1 = torch.arange(1,6, dtype=torch.int32, device=device)
t2 = torch.ones(15, dtype=torch.float32, device=device).view(3,5)
t_res = t2.matmul(t1.float()).to(device='cpu')

In [ ]:
t_res

tensor([15., 15., 15.])

Task 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
targets = torch.rand(1000).to(device)
predict = torch.rand(1000).to(device)

In [ ]:
Q = torch.sum(torch.pow((targets - predict), 2)) / 1000

In [ ]:
Q = Q.to('cpu')

In [ ]:
Q

tensor(169.7121)

Task 3

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

w = torch.rand(1, 16).to(device) # вектор-строка
X = torch.rand(16, 16).to(device) # матрица 16 x 16
I = torch.eye(16).to(device) # единичная матрица 16 x 16
lm = 0.5 # значение лямбда

In [ ]:
Q = w.matmul((X.add(I.mul(lm)))).matmul(w.transpose(0,1))

In [ ]:
Q.to('cpu')

tensor([[27.8480]])

Task 4

In [1]:
import torch
# все эти переменные в программе не менять, только тензоры разместить на device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

W = torch.rand(3) * 10 - 5
bias = torch.rand(1) * 100 - 50

batch_size = 8 # размер мини-батча
X = torch.empty(batch_size, 3).normal_(mean=-1.0, std=4.0).to(device)
predict = (X.to(device).mv(W.to(device)) + bias.to(device)).sigmoid()